# Effective DOF test

## Import package

In [1]:
import numpy as np
import netCDF4 as nc

## Load data

In [2]:
# Load MPAS data
## dimension
import time


dims: dict[str, np.ndarray] = dict()

temp: dict[str, np.ndarray] = dict()


## variables
with nc.Dataset("/work/b11209013/2024_Research/MPAS/merged_data/CNTL/theta.nc", "r") as f:
    for key in f.dimensions.items():
        dims[key[0]] = f.variables[key[0]][:]

    lat_lim: list[int] = np.where((dims["lat"][:] >= -5) & (dims["lat"][:] <= 5))[0]

    temp["CNTL"] = (f.variables["theta"][:, :, lat_lim, :].mean(axis=2) * (1000/dims["lev"])[None, :, None]**(-0.286)) - 273.15

with nc.Dataset("/work/b11209013/2024_Research/MPAS/merged_data/NCRF/theta.nc", "r") as f:
    temp["NCRF"] = (f.variables["theta"][:, :, lat_lim, :].mean(axis=2) * (1000/dims["lev"])[None, :, None]**(-0.286)) - 273.15

# load events
lon_ref: dict[str, np.ndarray] = dict()
time_ref: dict[str, np.ndarray] = dict()

for key in temp.keys():
    lon_ref[key], time_ref[key] = np.load(
        f"/home/b11209013/2024_Research/MPAS/Composite/Q1_event_sel/Q1_sel/{key}.npy"
    )


## Event selection

### Time interval setting

In [5]:
time_itv: dict[str, np.ndarray] = dict()

for key in temp.keys():
    time_itv[key] = np.array([
        np.linspace(time_ref[key][i]-12, time_ref[key][i]+12, 25)
        for i in range(time_ref[key].shape[0])
    ])

# shape: (events, time_length)

In [7]:
temp_sel: dict[str, np.ndarray] = dict()

for key in temp.keys():
    temp_sel[key] = np.array([
        temp[key][time_itv[key][i].astype(int), :, lon_ref[key][i].astype(int)].T
        for i in range(time_ref[key].shape[0])
    ])

# shape: (events, lev, time_length)

## Compute effective DOF

In [ ]:
for key in temp_sel.keys():
    for i in range(temp_sel[key].shape[0]):
        for j in range(temp_sel[key].shape[1]):
            temp_sel[key][i, j, :] = temp_sel[key][i, j, :] - temp_sel[key][i, j, 0]